In [1]:
import pandas as pd
import numpy as np

In [2]:
#to assign fips codes off of abbreviation
state_codes = {
    'WA': '53', 'DE': '10', 'DC': '11', 'WI': '55', 'WV': '54', 'HI': '15',
    'FL': '12', 'WY': '56', 'PR': '72', 'NJ': '34', 'NM': '35', 'TX': '48',
    'LA': '22', 'NC': '37', 'ND': '38', 'NE': '31', 'TN': '47', 'NY': '36',
    'PA': '42', 'AK': '02', 'NV': '32', 'NH': '33', 'VA': '51', 'CO': '08',
    'CA': '06', 'AL': '01', 'AR': '05', 'VT': '50', 'IL': '17', 'GA': '13',
    'IN': '18', 'IA': '19', 'MA': '25', 'AZ': '04', 'ID': '16', 'CT': '09',
    'ME': '23', 'MD': '24', 'OK': '40', 'OH': '39', 'UT': '49', 'MO': '29',
    'MN': '27', 'MI': '26', 'RI': '44', 'KS': '20', 'MT': '30', 'MS': '28',
    'SC': '45', 'KY': '21', 'OR': '41', 'SD': '46', 'GU': '66', 'VI': '78'}

In [3]:
#load in health data
df = pd.read_csv("/data/discover/Data/Health and Nutrition/health_and_nutrition_measures.csv",dtype={
    'year':str,'state_fips': object,'county_fips':object,'tract_fips':object,'GEO_ID':str})

#variable to match each each, due to changes some years
food_insecurity_var_dict={"2021":'Overall Food Insecurity Rate',
                         "2020":'Overall Food Insecurity Rate',
                         "2019":"Overall Food Insecurity Rate",
                         "2018":"2018 Food Insecurity Rate",
                         "2017":"2017 Food Insecurity Rate",
                         "2016":"2016 Food Insecurity Rate",
                         "2015":"2015 Food Insecurity Rate"}

food_insecurity_var_dict_state={"2021":'Overall Food Insecurity Rate ',
                         "2020":'Overall Food Insecurity Rate ',
                         "2019":'Overall Food Insecurity Rate ',
                         "2018":"2018 Food Insecurity Rate",
                         "2017":"2017 Food Insecurity Rate",
                         "2016":"2016 Food Insecurity Rate",
                         "2015":"2015 Food Insecurity Rate"}

#filepath = filepath string
#year is to define the year for the data we are merging
#sheat number, tab on the xlsx that the data is on
#header defines row that the column names are in
#geo set to state or county
def food_insercure_merge(filepath,year,sheat_number,header_var,geo):
    if geo == "county":
        food_insecurity_var = food_insecurity_var_dict[year]
    if geo == "state":
        food_insecurity_var = food_insecurity_var_dict_state[year]
        
    merge_frame_1 = pd.read_excel(filepath,sheet_name=sheat_number,header=header_var,dtype={'FIPS':str,'Year':str})
    #Have to add zeroes to ones below 10 like 01001 for example
    if geo == "county":
        merge_frame_1['FIPS'] = merge_frame_1['FIPS'].apply(lambda x: x if len(x) == 5 else '0'+x)
    if geo =="state":
        merge_frame_1['FIPS'] = merge_frame_1['FIPS'].apply(lambda x: x if len(x) == 2 else '0'+x)
    
    # make percentage a whole number
    merge_frame_1[food_insecurity_var] = merge_frame_1[food_insecurity_var]*100
    
    #below use existing year column or creates new one if not existing
    if 'Year' in merge_frame_1.columns:
        merge_frame_1 = merge_frame_1[merge_frame_1['Year'] == year]
    else:
        merge_frame_1['Year'] = year
    
    #renames columns
    merge_frame_1.rename(columns={'FIPS':'GEO_ID', 'Year':'year',food_insecurity_var:'Overall Food Insecurity Rate'}, inplace=True)

    #keep only relevant columns
    merge_frame_1 = merge_frame_1[['GEO_ID','year','Overall Food Insecurity Rate']]
    
    return merge_frame_1
    
#get county dataframes
df_2021_cnty = food_insercure_merge("/data/discover/Data/Health and Nutrition/5_Food_Insecure/MMG2023_2019-2021_Data_ToShare.xlsx","2021",1,0,"county")
df_2020_cnty = food_insercure_merge("/data/discover/Data/Health and Nutrition/5_Food_Insecure/MMG2023_2019-2021_Data_ToShare.xlsx","2020",1,0,"county")
df_2019_cnty = food_insercure_merge("/data/discover/Data/Health and Nutrition/5_Food_Insecure/MMG2023_2019-2021_Data_ToShare.xlsx","2019",1,0,"county")
df_2018_cnty = food_insercure_merge("/data/discover/Data/Health and Nutrition/5_Food_Insecure/MMG2020_2018Data_ToShare.xlsx","2018",0,1,"county")
df_2017_cnty = food_insercure_merge("/data/discover/Data/Health and Nutrition/5_Food_Insecure/MMG2019_2017Data_ToShare.xlsx","2017",0,0,"county")
df_2016_cnty = food_insercure_merge("/data/discover/Data/Health and Nutrition/5_Food_Insecure/MMG2018_2016Data_ToShare.xlsx","2016",0,0,"county")
df_2015_cnty = food_insercure_merge("/data/discover/Data/Health and Nutrition/5_Food_Insecure/MMG2017_2015Data_ToShare.xlsx","2015",0,0,"county")

#get state dataframes
df_2021_state = food_insercure_merge("/data/discover/Data/Health and Nutrition/5_Food_Insecure/MMG2023_2019-2021_Data_ToShare.xlsx","2021",3,0,"state")
df_2020_state = food_insercure_merge("/data/discover/Data/Health and Nutrition/5_Food_Insecure/MMG2023_2019-2021_Data_ToShare.xlsx","2020",3,0,"state")
df_2019_state = food_insercure_merge("/data/discover/Data/Health and Nutrition/5_Food_Insecure/MMG2023_2019-2021_Data_ToShare.xlsx","2019",3,0,"state")
df_2018_state = food_insercure_merge("/data/discover/Data/Health and Nutrition/5_Food_Insecure/MMG2020_2018Data_ToShare.xlsx","2018",2,1,"state")
df_2017_state = food_insercure_merge("/data/discover/Data/Health and Nutrition/5_Food_Insecure/MMG2019_2017Data_ToShare.xlsx","2017",2,0,"state")
df_2016_state = food_insercure_merge("/data/discover/Data/Health and Nutrition/5_Food_Insecure/MMG2018_2016Data_ToShare.xlsx","2016",2,0,"state")
df_2015_state = food_insercure_merge("/data/discover/Data/Health and Nutrition/5_Food_Insecure/MMG2017_2015Data_ToShare.xlsx","2015",2,0,"state")

#concatinate everything into a dataframe then left merge off of that dataframe
df_concat = pd.concat([df_2021_cnty,df_2020_cnty,df_2019_cnty,df_2018_cnty,df_2017_cnty,df_2016_cnty,df_2015_cnty,
                      df_2021_state,df_2020_state,df_2019_state,df_2018_state,df_2017_state,df_2016_state,df_2015_state])

df_final = df.merge(right=df_concat,on=['GEO_ID','year'],how='left')


/tmp/ipykernel_21858/282942215.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/data/discover/Data/Health and Nutrition/health_and_nutrition_measures.csv",dtype={


In [4]:
#to clean and merge 
#filepath = filepath string
#year is to define the year for the data we are merging
#sheat number, tab on the xlsx that the data is on
#header defines row that the column names are in
def food_access_merge(filepath,year,sheat_number,header_var):
    
    #pull in data
    merge_frame = pd.read_excel(filepath,sheet_name=sheat_number,header=header_var,dtype={'CensusTract':str,'LA1and10':bool})
    
    #create year column
    merge_frame['year'] = year
   
    #renames columns
    merge_frame.rename(columns={'CensusTract':'GEO_ID'}, inplace=True)
    
    #keep only relevant columns
    merge_frame = merge_frame[['GEO_ID','year','LA1and10']]
    
    return merge_frame


In [5]:
#create data frames for food access
df_2019_food_access = food_access_merge("/data/discover/Data/Health and Nutrition/6_Supermarket_dist/FoodAccessResearchAtlasData2019.xlsx","2019",2,0)
df_2015_food_access = food_access_merge("/data/discover/Data/Health and Nutrition/6_Supermarket_dist/FoodAccessResearchAtlasData2015.xlsx","2015",2,0)

#concatinate and left merge the dataframe to main one
df_concat_food_access = pd.concat([df_2019_food_access,df_2015_food_access])

df_final_2 = df_final.merge(df_concat_food_access,on=['GEO_ID','year'],how='left')


In [6]:
# for Behaviorial risk state table, cleans the data for merge
#filepath = filepath string
#header defines row that the column names are in
def Behavioral_Risk_state_merge(filepath,header_var):
        
    merge_frame = pd.read_csv(filepath,header=header_var,dtype={'Year':str})
    
    #Keep relevant years
    merge_frame = merge_frame[(merge_frame['Year'] == "2015") | (merge_frame['Year'] == "2016")|(merge_frame['Year'] == "2017")|
                              (merge_frame['Year'] == "2018")|(merge_frame['Year'] == "2019")|(merge_frame['Year'] == "2020")|
                              (merge_frame['Year'] == "2021")]
    
    #filter only to neeeded values
    merge_frame = merge_frame[merge_frame['Response'].isin(['Poor','Fair'])]
    
    merge_frame = merge_frame[merge_frame['Break_Out'].isin(['Overall'])]
    
    #map fips codes onto states
    merge_frame['GEO_ID'] = merge_frame['Locationabbr'].map(state_codes)
    
    #rename columns to match final file
    merge_frame.rename(columns={'Year':'year'}, inplace=True)
    
    #keep only needed fields
    merge_frame = merge_frame[['GEO_ID','year','Data_value']]
    
    #sums both % poor and fair health together, this matches county level data better
    merge_frame = merge_frame.groupby(['year','GEO_ID']).agg(Per_Fair_and_Poor_Health=('Data_value','sum'),
                                                            GEO_ID=('GEO_ID','first'),
                                                            year=('year','first'))
    
    return merge_frame

In [7]:
def Behavioral_Risk_county_merge(filepath,year,header_var):
    
    merge_frame = pd.read_csv(filepath,
                              header=header_var,
                              dtype={'Year':str, 'LocationID':str})
    merge_frame = merge_frame[merge_frame['Measure'].isin(["Fair or poor self-rated health status among adults aged >=18 years"])]
    merge_frame = merge_frame[merge_frame['Data_Value_Type'].isin(["Crude prevalence"])]
    merge_frame = merge_frame[merge_frame['Year'] == year]
    merge_frame.rename(columns={'Year':'year','Data_Value':'Per_Fair_and_Poor_Health','LocationID':'GEO_ID'}, inplace=True)
    merge_frame = merge_frame[['GEO_ID','year','Per_Fair_and_Poor_Health']]
    return merge_frame


In [8]:
#Get dataframe for state level data using function
Behavioral_Risk_state_df = Behavioral_Risk_state_merge("/data/discover/Data/Health and Nutrition/8_Self_Rated Health/State/Behavioral_Risk_Factor_Surveillance_System__BRFSS__Prevalence_Data__2011_to_present__20240502.csv",
          0)

#Get dataframes for county level data using function
df_2021_Behavioral_Risk_county = Behavioral_Risk_county_merge("/data/discover/Data/Health and Nutrition/8_Self_Rated Health/County/PLACES__Local_Data_for_Better_Health__County_Data_2023_release_20240502.csv",
                                                             "2021",0)
df_2020_Behavioral_Risk_county = Behavioral_Risk_county_merge("/data/discover/Data/Health and Nutrition/8_Self_Rated Health/County/PLACES__Local_Data_for_Better_Health__County_Data_2022_release_20240502.csv",
                                                             "2020",0)
df_2019_Behavioral_Risk_county = Behavioral_Risk_county_merge("/data/discover/Data/Health and Nutrition/8_Self_Rated Health/County/PLACES__Local_Data_for_Better_Health__County_Data_2021_release_20240502.csv",
                                                             "2019",0)

#concatinate into one dataframe
df_concat_Behavioral_Risk = pd.concat([Behavioral_Risk_state_df,df_2021_Behavioral_Risk_county,df_2020_Behavioral_Risk_county,df_2019_Behavioral_Risk_county])

#left merge and output final csv
df_final_3 = df_final_2.merge(df_concat_Behavioral_Risk,on=['GEO_ID','year'],how='left')

df_final_3.to_csv("/data/discover/Data/Health and Nutrition/health_and_nutrition_measures_final_output.csv")

/tmp/ipykernel_21858/1563365460.py:6: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  merge_frame = pd.read_csv(filepath,header=header_var,dtype={'Year':str})
